In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish_validation_train


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

# from src.ad_hmm import sku_predict
# from src.ad_stl_model import stl_model
from src.utils import *

In [3]:
%load_ext autoreload 
%autoreload 

### Subset SKUs

In [4]:
validation_data = pd.read_excel("../../data/Validation_Data.xlsx")
validation_data.shape

(3773, 12)

In [5]:
validation_data.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,Forecasted Units
0,23,2022-08-06,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,?,Moderate,3,NaN
1,100,2022-08-02,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,16.99,?,16.99,Fully-Stocked,2,NaN
2,121,2022-08-06,MISC ACCY,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,24.99,?,?,Fully-Stocked,6,NaN
3,207,2022-08-03,OTR MIC,OTR MICROWAVE,ML - Kitchen,VP - Major Appliances,499.99,?,?,Fully-Stocked,0,NaN
4,416,2022-08-05,AMPS,CAR STEREO,ML - Connected Car,VP - Electrify & Car,269.99,?,?,Fully-Stocked,9,NaN


In [6]:
validation_data.SALES_DATE.value_counts()

2022-08-06    539
2022-08-02    539
2022-08-03    539
2022-08-05    539
2022-08-04    539
2022-08-07    539
2022-08-01    539
Name: SALES_DATE, dtype: int64

In [7]:
validation_sku_list = validation_data.Encoded_SKU_ID.unique()

### RMSE final results

In [8]:
rmse_results = pd.read_excel("../../results/rmse_all_preds_jan_17.xlsx",index_col=0)
rmse_results.head()

,rmse_du_15,mean_du_15,pct_rmse_15,rmse_du_30,mean_du_30,pct_rmse_30,rmse_du_45,mean_du_45,pct_rmse_45,rmse_du_60,...,pct_rmse_prophet,rmse_du_null,mean_du_null,pct_rmse_null,rmse_du_holt,mean_du_holt,pct_rmse_holt,min_rmse,argmin_period,min_period
Encoded_SKU_ID,,,,,,,,,,,,,,,,,,,,,
82,4.722325,0.428571,11.018759,4.840344,0.428571,11.294136,1.102108,0.428571,2.571586,1.419381,...,5.362264,3.605551,0.428571,8.412953,9.753826,0.428571,22.758928,0.911043,5,rmse_du_max
568,4.097124,4.285714,0.955996,3.447670,4.285714,0.804456,3.652836,4.285714,0.852329,3.871231,...,0.595404,5.070926,4.285714,1.183216,3.024439,4.285714,0.705702,2.551733,8,rmse_du_prophet
317,3.766488,0.857143,4.394235,1.362770,0.857143,1.589899,1.784957,0.857143,2.082450,2.104417,...,2.297158,1.069045,0.857143,1.247219,2.363641,0.857143,2.757581,1.069045,9,rmse_du_null
95,3.668447,1.285714,2.853236,1.698949,1.285714,1.321405,3.991151,1.285714,3.104228,3.931648,...,1.743961,1.309307,1.285714,1.018350,1.720612,1.285714,1.338254,1.309307,9,rmse_du_null
340,3.668447,0.714286,5.135825,2.077172,0.714286,2.908041,2.329929,0.714286,3.261901,1.558387,...,0.970033,0.925820,0.714286,1.296148,1.092959,0.714286,1.530142,0.692881,8,rmse_du_prophet


In [9]:
rmse_results["min_period"].value_counts()

rmse_du_null       132
rmse_du_prophet     80
rmse_du_STL         78
rmse_du_max         40
rmse_du_30          34
rmse_du_holt        34
rmse_du_60          34
rmse_du_15          32
rmse_du_MSTL        27
rmse_du_45          27
rmse_du_90          27
Name: min_period, dtype: int64

In [10]:
rmse_results["min_period"].value_counts().sum()

545

In [11]:
set(validation_sku_list) - set(rmse_results.index)

{76, 147, 157, 358, 413, 485, 569}

In [12]:
add_all = set(validation_sku_list) - set(rmse_results.index)

In [13]:
sku_period_dict = rmse_results.reset_index().groupby("min_period")["Encoded_SKU_ID"].apply(set)

In [14]:
sku_period_dict1 = {}
for model_type in sku_period_dict.index:
    a = set(sku_period_dict[model_type])
    b = set(validation_sku_list)
#     print (len(a-b),len(b-a))
    c = a.intersection(b)
    d = c.union(add_all)
    print (model_type,len(a),len(a-b),len(d))
    sku_period_dict1[model_type] = list(d)

rmse_du_15 32 2 37
rmse_du_30 34 2 39
rmse_du_45 27 2 32
rmse_du_60 34 2 39
rmse_du_90 27 0 34
rmse_du_MSTL 27 0 34
rmse_du_STL 78 3 82
rmse_du_holt 34 0 41
rmse_du_max 40 0 47
rmse_du_null 132 2 137
rmse_du_prophet 80 0 87


In [15]:
import pickle

# a = {'hello': 'world'}

with open('../../results/final_sku_model_map.pkl', 'wb') as handle:
    pickle.dump(sku_period_dict1, handle, protocol=4)